In [ ]:
#setup
!pip install PyPDF2
!pip install nltk
!pip install gensim
!pip install nmslib
!pip install sentence-transformers
!pip install tensorflow
!pip install tensorflow_hub
!pip install pickle-mixin

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
# data collection
import PyPDF2
filename = 'C:\\Users\\syuhh\\corpus_textbook\\textbook_combined.pdf'
pdf_file = open(filename, 'rb')
ind_textbook = PyPDF2.PdfReader(pdf_file)
total_pages = len(ind_textbook.pages)

count = 0
rawtext  = ''

# loop through to read each page from the pdf file
while(count < total_pages):
    # Get the specified number of pages in the document
    textbook_page  = ind_textbook.pages[count]
    # Process the next page
    count += 1
    # Extract the text from the page
    rawtext += textbook_page.extract_text()

with open('C:\\Users\\syuhh\\corpus_textbook\\rawtext.txt', 'w', encoding="utf-8") as f:
    f.write(rawtext)

In [ ]:
#data cleansing
f = open('C:\\Users\\syuhh\\corpus_textbook\\rawtext_clean.txt', 'r', encoding="utf-8")
rawtext = f.read()
import re
#pattern = r"-\nI[A-Z][a-z]"
#pattern = r"c[1-2][0-9]\.fm.*\n"
#pattern = r"-\n"
#pattern = r"([a-z])\n([A-Z])"
#pattern = r"([a-z])+\s*\n([A-Z])"
pattern = r"\.\s{1,}([a-z])"
replacement = r"\1.\n\2"
result = re.sub(pattern, replacement, rawtext)

with open('C:\\Users\\syuhh\\corpus_textbook\\rawtext_clean.txt', 'w', encoding="utf-8") as f:
    f.write(result)

In [1]:
# text preprocessing
import nltk
import string
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

def preprocess(filename):
    f = open(filename,'r',encoding="utf8")
    text = f.read()
    #Lowercasing
    text_l = text.lower()
    # Numbers removal
    text_ld = "".join([c for c in text_l if not c.isdigit()])
    #Tokenisation + Stopwords removal + Lemmatization
    stop_words = set(stopwords.words('english'))
    exc =["if","for","do","while","same","all","where","how","when","what","which","who"]
    for w in exc:
        stop_words.remove(w)
    lemmatizer = WordNetLemmatizer()
    data = []
    sentences = []
    # iterate through each sentence in the file
    for i in sent_tokenize(text_ld):
        temp = []
        temp2 = []
        # tokenize the sentence into words
        for j in word_tokenize(i):
            if j not in string.punctuation:
                temp2.append(j)
            #Punctuation and non-alphanumeric character removal
                if j not in stop_words:
                    j = lemmatizer.lemmatize(j)
                    temp.append(j)
        sentences.append(" ".join(temp2))
        data.append(temp)
    #Named entity recognition no run
    #Stemming no run
    return data, sentences

#data, sentences = preprocess('/content/drive/MyDrive/Colab Notebooks/rawtext.txt')
data, sentences = preprocess("C:\\Users\\syuhh\\corpus_textbook\\rawtext_clean.txt")

In [2]:
print(data[:10])
print(len(data))

[['preface'], ['textbook', 'began', 'set', 'lecture', 'note', 'for', 'first-year', 'undergraduate', 'software', 'engineering', 'course'], ['course', 'run', 'week', 'semester', 'two', 'lecture', 'week'], ['intention', 'text', 'cover', 'topic', 'c', 'programming', 'language', 'introductory', 'software', 'design', 'sequence', 'lecture', 'course', 'material', 'chapter', 'well', 'served', 'two', 'lecture', 'apiece'], ['ample', 'cross-referencing', 'indexing', 'provided', 'make', 'text', 'serviceable', 'reference', 'complete', 'work', 'recommended'], ['particular', 'for', 'practicing', 'programmer', 'best', 'available', 'tutorial', 'reference', 'kernighan', 'ritchie', 'best', 'in-depth', 'reference', 'harbison', 'steele'], ['influence', 'two', 'work', 'text', 'readily', 'apparent', 'throughout'], ['what', 'set', 'book', 'apart', 'introductory', 'c-programming', 'text', 'strong', 'emphasis', 'software', 'design'], ['like', 'text', 'present', 'core', 'language', 'syntax', 'semantics', 'also', 

In [3]:
print(sentences[:5])
print(len(sentences))

['preface', 'this textbook began as a set of lecture notes for a first-year undergraduate software engineering course in', 'the course was run over a week semester with two lectures a week', 'the intention of this text is to cover topics on the c programming language and introductory software design in sequence as a lecture course with the material in chapters and well served by two lectures apiece', 'ample cross-referencing and indexing is provided to make the text a serviceable reference but more complete works are recommended']
39633


In [2]:
# word2vec cbow
import gensim
from gensim.models import Word2Vec

cbow_model = gensim.models.Word2Vec(data, min_count = 2, vector_size = 5000, 
                                    window = 4, hs = 1, negative=0)

print(cbow_model.wv.similarity('iteration','loop'))
similar_words1 = cbow_model.wv.most_similar('loop', topn=10)
print(similar_words1)

0.5460193
[('do-while', 0.7650426626205444), ('statement', 0.7404494881629944), ('continue', 0.7077022194862366), ('executed', 0.681331992149353), ('break', 0.6717720627784729), ('body', 0.671444296836853), ('enclosing', 0.6347253322601318), ('transfer', 0.6344325542449951), ('while', 0.6291719675064087), ('condition', 0.6276076436042786)]


In [5]:
print("loop -->")
similar_words1 = cbow_model.wv.most_similar('loop', topn=10)
print(similar_words1)
print("\n printf -->")
similar_words2 = cbow_model.wv.most_similar('printf', topn=10)
print(similar_words2)

loop -->
[('statement', 0.7829611897468567), ('do-while', 0.7455766797065735), ('executed', 0.7041654586791992), ('continue', 0.6850815415382385), ('break', 0.6756772398948669), ('condition', 0.6600749492645264), ('enclosing', 0.6535517573356628), ('body', 0.6341366171836853), ('brace', 0.6253629922866821), ('while', 0.6179232597351074)]

 printf -->
[('``', 0.6781397461891174), ('d/', 0.6452648639678955), ('rb', 0.6238480806350708), ('deck', 0.6188544034957886), ('\\n', 0.6171674728393555), ('p.on_hand', 0.612030029296875), ('world', 0.5959655046463013), ('n.kind', 0.5938659310340881), ("''", 0.5849100947380066), ('d\\n', 0.5782233476638794)]


In [6]:
# word2vec skipgram
skipgram_model = gensim.models.Word2Vec(data, min_count = 2, vector_size = 5000, 
                                        window = 4, sg = 1, hs = 1, negative=0)

print(skipgram_model.wv.similarity('iteration','loop'))
similar_words1 = skipgram_model.wv.most_similar('loop', topn=10)
print(similar_words1)

0.6550745
[('do-while', 0.6988455653190613), ('executed', 0.6893923282623291), ('body', 0.6795340180397034), ('iteration', 0.6550743579864502), ('forever', 0.6385603547096252), ('transfer', 0.6347364783287048), ('continue', 0.632845938205719), ('statement', 0.6039971709251404), ('mistaken', 0.6009771227836609), ('enclosing', 0.5964893102645874)]


In [7]:
print("loop -->")
similar_words1 = skipgram_model.wv.most_similar('loop', topn=10)
print(similar_words1)
print("\n printf -->")
similar_words2 = skipgram_model.wv.most_similar('printf', topn=10)
print(similar_words2)

loop -->
[('do-while', 0.6988455653190613), ('executed', 0.6893923282623291), ('body', 0.6795340180397034), ('iteration', 0.6550743579864502), ('forever', 0.6385603547096252), ('transfer', 0.6347364783287048), ('continue', 0.632845938205719), ('statement', 0.6039971709251404), ('mistaken', 0.6009771227836609), ('enclosing', 0.5964893102645874)]

 printf -->
[('``', 0.6267349123954773), ('i/j', 0.6042519211769104), ('d\\n', 0.5987091064453125), ('order.amount', 0.5899890661239624), ('s.flag', 0.5711936950683594), ('quarter', 0.5668281316757202), ('nickel', 0.5653830170631409), ('km\\n', 0.5620003342628479), ('ld\\n', 0.5617414116859436), ('prime\\n', 0.560014009475708)]


In [8]:
# bert + nmslib
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

words = [w for s in data for w in s]
df = pd.DataFrame({"words": words})
word_data = list(df["words"].to_list())

#Sentences are encoded by calling model.encode()
encoding = model.encode(word_data)

data_encoding = np.array(encoding)
np.save("encoding.npy", data_encoding)
embedding= np.load("encoding.npy")

In [9]:
#Visualizing The Embedding Of Words
embedding= np.load("encoding.npy")

words = [w for s in data for w in s]
df = pd.DataFrame({"words": words})

vector_list = []
for emb in embedding:
    vector_list.append(emb)

vect = np.array(vector_list)

df = pd.DataFrame({"vector":vector_list, "title":df["words"]})
df = df.reset_index()
df.drop(columns=["index"],inplace=True)
df

,vector,title
0,"[-0.010352628, 0.07233449, 0.024498053, 0.0570...",preface
1,"[-0.039159015, 0.025386218, -0.043094616, 0.07...",textbook
2,"[-0.012888496, -0.024884595, -0.011723027, 0.0...",began
3,"[-0.017035225, 0.051864002, -0.046009693, 0.03...",set
4,"[-0.015049514, 0.044220474, -0.052697603, -0.0...",lecture
...,...,...
320617,"[0.08597011, 0.06067221, -0.06504023, -0.01759...",minimum
320618,"[-0.086003326, 0.00682386, 0.004462829, 0.0483...",n
320619,"[-0.086003326, 0.00682386, 0.004462829, 0.0483...",n
320620,"[0.04064646, 0.043968488, -0.017024042, 0.0127...",normalized


In [10]:
import nmslib
from nltk.stem import PorterStemmer

ps = PorterStemmer()

class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[0]
        self.vectors = vectors.astype("float32")
        self.labels = labels

    def build(self):
        self.index = nmslib.init(space='cosinesimil',method='hnsw')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({"post":2})

    def query(self,vector,k):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]],[round(i,2) for i in indices[1]]

# Building The Search Index
nm_index = NMSLIBIndex(vect,df["title"])
nm_index.build()

In [11]:
# Getting The Nearest Neighbors For Each Word (most similar words)
# looking for n/K NNs for each word
K = 10

pd_dict= {"main_word":[], "similar_word":[], "distance":[], "group":[]}
group = 0

for vector,label in zip(df['vector'], df['title']):
    queries_nm, distances_nm = nm_index.query(vector,K)
    for q,d in zip(queries_nm, distances_nm):
            if d>0.1:
                pd_dict["main_word"].append(label)
                pd_dict["similar_word"].append(q)
                pd_dict["distance"].append(d)
                pd_dict["group"].append(group)

    group += 1

In [12]:
# Summarizing The Results

frame = pd.DataFrame(pd_dict)
frame.drop_duplicates(subset="similar_word",inplace=True)

syns = []
import collections
t = collections.defaultdict(list)

for g in set(frame["group"].to_list()):
    rows = frame[frame["group"] == g]["similar_word"].to_list()
    rows2 = frame[frame["group"] == g]["main_word"].to_list()[0]
    t[rows2] = rows
    rows.append(rows2)
    syns.append(rows)
    
# similar word for a given input


In [13]:
frame.head(50)

,main_word,similar_word,distance,group
0,preface,introduction,0.33,0
7,textbook,need,0.67,1
17,began,attempted,0.53,2
18,began,attempting,0.55,2
27,lecture,instructor,0.36,4
29,lecture,teaching,0.37,4
30,lecture,class,0.37,4
33,note,’,0.56,5
43,first-year,beginning,0.40,7
52,undergraduate,college,0.27,8


In [14]:
print("\n loop -->")
for group in syns:
    for word in group:
        if word =="loop":
              print(group)

print("\n printf -->")
for group in syns:
    for word in group:
        if word =="printf":
              print(group)


 loop -->
['loop', 'indexing']

 printf -->
['printf', 'printing']
['char', 'printf']


In [15]:
# fasttext
from gensim.models import FastText

fasttext_model = FastText(vector_size=2000, window=4, min_count=2)  # instantiate
fasttext_model .build_vocab(data)
fasttext_model .train(data, total_examples=len(data), epochs=100)  # train

print(fasttext_model.wv.similarity('iteration','loop'))
similar_words1 = fasttext_model.wv.most_similar(positive=['iteration'], topn = 10)
print(similar_words1)

0.3045722
[('alteration', 0.8559949994087219), ('ration', 0.7898642420768738), ('iteration-statement', 0.770746111869812), ('zation', 0.7097257971763611), ('consideration', 0.6981011033058167), ('ation', 0.6961163878440857), ('operation', 0.6848737597465515), ('generation', 0.6745608448982239), ('duration', 0.6555023193359375), ('configuration', 0.6542657017707825)]


In [16]:
print("loop -->")
similar_words1 = fasttext_model.wv.most_similar(positive=['loop'], topn = 10)
print(similar_words1)
print("\n printf -->")
similar_words2 = fasttext_model.wv.most_similar(positive=['printf'], topn = 10)
print(similar_words2)

loop -->
[('timeloop', 0.7359610199928284), ('loop_done', 0.729006826877594), ('looping', 0.6567798852920532), ('loose', 0.3977554440498352), ('ifstatement', 0.38792967796325684), ('substatement', 0.38574081659317017), ('do-while', 0.3837915062904358), ('op', 0.37323707342147827), ('statement', 0.3693470060825348), ('iteration-statement', 0.36146387457847595)]

 printf -->
[('vprintf', 0.8317158818244934), ('v…printf', 0.8176165223121643), ('wprintf', 0.8175871968269348), ('ofprintf', 0.8169824481010437), ('…printf', 0.8120958805084229), ('vwprintf', 0.8064144849777222), ('rintf', 0.7987238168716431), ('sprintf', 0.790643036365509), ('vsprintf', 0.786302387714386), ('fwprintf', 0.7817254066467285)]


In [6]:
# pretrained model library

import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


import gensim.downloader

word2vec_pretrained = gensim.downloader.load('word2vec-google-news-300')

print(word2vec_pretrained.similarity('iteration','loop'))
similar_words1 = word2vec_pretrained.most_similar(positive=['iteration'], topn = 10)
print(similar_words1)

In [18]:
# smaller size of Word2Vec pretrained
import gensim
from nltk.data import find
word2vec_smaller = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_pretrained = gensim.models.KeyedVectors.load_word2vec_format(word2vec_smaller, binary=False)

#print(word2vec_pretrained.similarity('iteration','loop'))
#similar_words1 = word2vec_pretrained.most_similar(positive=['iteration'], topn = 10)
#print(similar_words1)

In [3]:
#pretrained Word2Vec fine-tuning
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

word2vec_tuned = Word2Vec(vector_size=100, min_count=2)
word2vec_tuned.build_vocab(data)

#word2vec_pretrained_path = "C:\\Users\\syuhh\\gensim-data\\word2vec-google-news-300
#\\word2vec-google-news-300.gz"
#unable to run above as the resoures needed is too large
# a pruned version is used (vocab contains 43981 words)
word2vec_pretrained_path = "C:\\Users\\syuhh\\AppData\\Roaming\\nltk_data\\models\\word2vec_sample\\pruned.word2vec.txt"
word2vec_pretrained = KeyedVectors.load_word2vec_format(word2vec_pretrained_path, binary=False)

# Add the pre-trained model vocabulary
import numpy as np
word2vec_tuned.build_vocab([list(word2vec_pretrained.index_to_key)], update=True)
total_examples = word2vec_tuned.corpus_count

# Load the pre-trained models embeddings
# note: if a word doesn't exist in the pre-trained vocabulary 
# then it is left as is in the original model
word2vec_tuned.wv.vectors_lockf = np.ones(len(word2vec_tuned.wv), dtype=np.float16)
word2vec_tuned.wv.intersect_word2vec_format(word2vec_pretrained_path, binary=False, lockf=1.0)

word2vec_tuned.train(data, total_examples=total_examples, epochs=word2vec_tuned.epochs)

ValueError: incompatible vector size 300 in file C:\Users\syuhh\AppData\Roaming\nltk_data\models\word2vec_sample\pruned.word2vec.txt

In [ ]:
print("\n loop -->")
similar_words1 = word2vec_tuned.wv.most_similar(positive=['loop'], topn = 10)
print(similar_words1)
print("\n printf -->")
similar_words2 = word2vec_tuned.wv.most_similar(positive=['printf'], topn = 10)
print(similar_words2)

In [7]:
#pretrained GloVe model
import gensim
import gensim.downloader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#run below for first run
glove_pretrained = gensim.downloader.load('glove-wiki-gigaword-300')
glove_pretrained_path = "C:\\Users\\syuhh\\gensim-data\\glove-wiki-gigaword-300\\glove-wiki-gigaword-300.gz"
glove_pretrained = KeyedVectors.load_word2vec_format(glove_pretrained_path, binary=False)

print(glove_pretrained.similarity('iteration','loop'))
similar_words1 = glove_pretrained.most_similar(positive=['iteration'], topn = 10)
print(similar_words1)

[==================================================] 100.0% 376.1/376.1MB downloaded
0.3161096
[('iterations', 0.645356297492981), ('algorithm', 0.5090993642807007), ('iterative', 0.4905813932418823), ('computed', 0.4452853500843048), ('approximation', 0.429524302482605), ('computation', 0.4205939769744873), ('codice_1', 0.4104224443435669), ('instalment', 0.405374139547348), ('permutation', 0.4050779342651367), ('computes', 0.40470555424690247)]


In [10]:
#pretrained GloVe model fine-tuning
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

glove_tuned = Word2Vec(vector_size=300, min_count=2)
glove_tuned.build_vocab(data)

glove_pretrained_path = "C:\\Users\\syuhh\\gensim-data\\glove-wiki-gigaword-300\\glove-wiki-gigaword-300.gz"
glove_pretrained = KeyedVectors.load_word2vec_format(glove_pretrained_path, binary=False)

# Add the pre-trained model vocabulary
import numpy as np
glove_tuned.build_vocab([list(glove_pretrained.index_to_key)], update=True)
total_examples = glove_tuned.corpus_count

# Load the pre-trained models embeddings
# note: if a word doesn't exist in the pre-trained vocabulary 
# then it is left as is in the original model
glove_tuned.wv.vectors_lockf = np.ones(len(glove_tuned.wv), dtype=np.float32)
glove_tuned.wv.intersect_word2vec_format(glove_pretrained_path, binary=False, lockf=1.0)

glove_tuned.train(data, total_examples=total_examples, epochs=glove_tuned.epochs)

(1351625, 1603110)

In [11]:
print("\n loop -->")
similar_words1 = glove_tuned.wv.most_similar(positive=['loop'], topn = 10)
print(similar_words1)
print("\n printf -->")
similar_words2 = glove_tuned.wv.most_similar(positive=['printf'], topn = 10)
print(similar_words2)


 loop -->
[('intx', 0.7398110032081604), ('x++', 0.7298818230628967), ('do-while', 0.7262066602706909), ('x=', 0.7172942757606506), ('z=', 0.7151269912719727), ('expr', 0.711052656173706), ('y=', 0.7076507210731506), ('++i', 0.6921600699424744), ('=t', 0.6815797090530396), ('c=getchar', 0.6805795431137085)]

 printf -->
[('type\\n', 0.438312828540802), ('variadic', 0.39523354172706604), ('ld\\n', 0.3732050657272339), ('—which', 0.3637981414794922), ('s\\n', 0.3592255413532257), ('denom', 0.35905635356903076), ('d\\n', 0.3566039204597473), ('function', 0.355427622795105), ('.s\\n', 0.3521178364753723), ('f\\n', 0.34731367230415344)]


In [12]:
#pretrained fasttext
#pretrained GloVe model
import gensim
import gensim.downloader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# run below for first run
#fasttext_pretrained = gensim.downloader.load('fasttext-wiki-news-subwords-300')
fasttext_pretrained_path = "C:\\Users\\syuhh\\gensim-data\\fasttext-wiki-news-subwords-300\\fasttext-wiki-news-subwords-300.gz"
fasttext_pretrained = KeyedVectors.load_word2vec_format(fasttext_pretrained_path, binary=False)

print(fasttext_pretrained.similarity('iteration','loop'))
similar_words = fasttext_pretrained.most_similar(positive=['iteration'], topn = 10)
print(similar_words1)
similar_words2 = fasttext_pretrained.most_similar(positive=['printf'], topn = 10)
print(similar_words2)

[==================================================] 100.0% 958.5/958.4MB downloaded
0.5485749
[('intx', 0.7398110032081604), ('x++', 0.7298818230628967), ('do-while', 0.7262066602706909), ('x=', 0.7172942757606506), ('z=', 0.7151269912719727), ('expr', 0.711052656173706), ('y=', 0.7076507210731506), ('++i', 0.6921600699424744), ('=t', 0.6815797090530396), ('c=getchar', 0.6805795431137085)]
[('fprintf', 0.8410983085632324), ('snprintf', 0.8028349876403809), ('sprintf', 0.7848883271217346), ('scanf', 0.7585453987121582), ('println', 0.7295418977737427), ('stdio.h', 0.709204912185669), ('memcpy', 0.6922029256820679), ('stdio', 0.6906760334968567), ('strcpy', 0.6871660947799683), ('strncpy', 0.6685115098953247)]


In [13]:
#pretrained fasttext model fine-tuning
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

fasttext_tuned = Word2Vec(vector_size=300, min_count=2)
fasttext_tuned.build_vocab(data)

fasttext_pretrained_path = """C:\\Users\\syuhh\\gensim-data\\
fasttext-wiki-news-subwords-300\\fasttext-wiki-news-subwords-300.gz"""
fasttext_pretrained = KeyedVectors.load_word2vec_format(fasttext_pretrained_path, binary=False)

# Add the pre-trained model vocabulary
import numpy as np
fasttext_tuned.build_vocab([list(fasttext_pretrained.index_to_key)], update=True)
total_examples = fasttext_tuned.corpus_count

# Load the pre-trained models embeddings
# note: if a word doesn't exist in the pre-trained vocabulary 
# then it is left as is in the original model
fasttext_tuned.wv.vectors_lockf = np.ones(len(fasttext_tuned.wv), dtype=np.float32)
fasttext_tuned.wv.intersect_word2vec_format(fasttext_pretrained_path, binary=False, lockf=1.0)

fasttext_tuned.train(data, total_examples=total_examples, epochs=fasttext_tuned.epochs)

(1352269, 1603110)

In [14]:
print("loop -->")
similar_words1 = fasttext_tuned.wv.most_similar(positive=['loop'], topn = 10)
print(similar_words1)
print("\n printf -->")
similar_words2 = fasttext_tuned.wv.most_similar(positive=['printf'], topn = 10)
print(similar_words2)

loop -->
[('x=', 0.9718918800354004), ('y=', 0.9707276225090027), ('z=', 0.9702528119087219), ('caselabel', 0.9666198492050171), ('expression', 0.9656054377555847), ('intn', 0.9637839794158936), ('statement', 0.9631353616714478), ('name-hiding', 0.9630696773529053), ('while', 0.9628068208694458), ('•functions', 0.9626761674880981)]

 printf -->
[("''", 0.9262217283248901), ('\\n', 0.925439178943634), ('d\\n', 0.9240949153900146), ('``', 0.9222123622894287), ('d\\t', 0.9096987247467041), ('world\\n', 0.9088935852050781), ('.f\\n', 0.9069238901138306), ('i++', 0.9065861105918884), ('e\\t', 0.9050455093383789), ('\\t', 0.9048459529876709)]


In [3]:
cbow_model.save("word_model.model")

In [ ]:
#sentence level embedding
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
#model2 = SentenceTransformer('all-mpnet-base-v2')

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Sentences are encoded by calling model.encode()
data_encoding = np.array(embeddings)
np.save("sents_embedding.npy", data_encoding)

In [ ]:
import numpy as np
from sentence_transformers.util import cos_sim

embeddings = np.load("sents_embedding.npy")

user_query = ['what is loop in C']
query_embedding = model.encode(user_query)

sim = np.zeros((len(user_query), len(sentences)))

for i in range(len(user_query)):
    for j in range(len(sentences)):
        sim[i,j] = cos_sim(query_embedding[i], embeddings[j])

most_related = sorted(zip(sim[0], sentences), reverse=True)[:3]
print(most_related)

In [ ]:
embeddings = np.load("sents_embedding.npy")

user_query = ['statements that executes for a specified number of repetitions']
query_embedding = model.encode(user_query)

sim = np.zeros((len(user_query), len(sentences)))

for i in range(len(user_query)):
    for j in range(len(sentences)):
        sim[i,j] = cos_sim(query_embedding[i], embeddings[j])

most_related = sorted(zip(sim[0], sentences), reverse=True)[:3]
print(most_related)

In [ ]:
embeddings = np.load("sents_embedding.npy")

user_query = ['loop']
query_embedding = model.encode(user_query)

sim = np.zeros((len(user_query), len(sentences)))

for i in range(len(user_query)):
    for j in range(len(sentences)):
        sim[i,j] = cos_sim(query_embedding[i], embeddings[j])

most_related = sorted(zip(sim[0], sentences), reverse=True)[:3]
print(most_related)

In [4]:
#ELMo
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

filtered = [sen for sen in sentences if len(sen.split()) > 1 and len(sen.split()) < 7]
# Load ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/2")

# Generate embeddings for example sentences
elmo_embeddings = elmo.signatures["default"](tf.constant(filtered))["default"]
elmo_vect = elmo_embeddings.numpy()

# Search string
search_string = ["loop"]
# Generate embeddings for search string
elmo_embeddings2 = elmo.signatures["default"](tf.constant(search_string))["default"]
search_vect = elmo_embeddings2.numpy()

# Reshape vectors for cosine similarity calculation
search_vect = search_vect.reshape(1, -1)
elmo_vect = elmo_vect.reshape(len(filtered), -1)

# Calculate cosine similarities
cosine_similarities = np.dot(search_vect, elmo_vect.T) / (
    np.linalg.norm(search_vect) * np.linalg.norm(elmo_vect, axis=1))

output = ""
for i in cosine_similarities.argsort()[0][-3:]:
    output += filtered[i] + "\n"
print(output)


loops while and fr
exit from loop
do-while loops



In [5]:
import pickle
with open('sentences.data', 'wb') as fp:
        pickle.dump(sentences, fp)
with open('sents_inELMo.data', 'wb') as fp:
        pickle.dump(filtered, fp)
with open('elmo_vect.data', 'wb') as fp:
        pickle.dump(elmo_vect, fp)

In [7]:
# Search string
search_string = ["program"]
# Generate embeddings for search string
elmo_embeddings2 = elmo.signatures["default"](tf.constant(search_string))["default"]
search_vect = elmo_embeddings2.numpy()

# Reshape vectors for cosine similarity calculation
search_vect = search_vect.reshape(1, -1)
elmo_vect = elmo_vect.reshape(len(filtered), -1)

# Calculate cosine similarities
cosine_similarities = np.dot(search_vect, elmo_vect.T) / (
    np.linalg.norm(search_vect) * np.linalg.norm(elmo_vect, axis=1))

output = ""
for i in cosine_similarities.argsort()[0][-3:]:
    output += filtered[i] + "\n"
print(output)

preprocessorc program
program termination
program quicksort

